# Notebook 3: Molecular Docking

In this notebook, you will **dock molecules into a protein binding site** using the computer program **AutoDock Vina**.

## What is molecular docking?

Think of docking like trying to fit a key into a lock:
- The **protein** is the lock (it has a specific binding pocket).
- The **molecule** is the key (it needs to fit into the pocket).

The docking software places the molecule inside the binding pocket and rotates/moves it around to find the best fit. It uses a smart search algorithm (not brute force!) to efficiently explore different orientations and flexes the molecule's rotatable bonds. For each pose it tries, it calculates a **score** (binding affinity) based on the physics and chemistry of the interaction.

## What will we do?

1. Re-download and prepare the ERα protein (so this notebook works on its own)
2. Generate 3D structures of drug molecules from SMILES
3. Dock them into the ERα binding site using AutoDock Vina
4. Visualize the docking results in 3D
5. Compare the docking scores with the ML predictions from Notebook 2

---

## Step 0: Install required packages

In [ ]:
%%capture
!pip install vina rdkit biopython py3Dmol
!apt-get install -y openbabel > /dev/null 2>&1

In [ ]:
import os
import math
import warnings
from math import isnan, isinf

import numpy as np
import pandas as pd
import py3Dmol

from vina import Vina
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from Bio.PDB import PDBList, PDBParser, PDBIO, Select, NeighborSearch, Structure, Model, Chain
from Bio.PDB.Polypeptide import is_aa

warnings.filterwarnings('ignore')

## Step 1: Download and prepare the protein

We repeat the protein preparation from Notebook 1 so this notebook is **self-contained** (you don't need to run Notebook 1 first).

In [ ]:
TARGET_PDB_ID = '3ERT'
PDB_CHAIN = 'A'
LIGAND_CODE = 'OHT'

In [ ]:
# Download the structure
pdb_list = PDBList(server='https://files.wwpdb.org', verbose=False)
downloaded_file = pdb_list.retrieve_pdb_file(TARGET_PDB_ID, file_format='pdb', pdir='.')
pdb_filename = f'{TARGET_PDB_ID}.pdb'
os.rename(downloaded_file, pdb_filename)

# Parse and extract chain A + ligand OHT
parser = PDBParser(QUIET=True)
full_structure = parser.get_structure(TARGET_PDB_ID, pdb_filename)

new_structure = Structure.Structure('extracted')
new_model = Model.Model(0)
new_chain = Chain.Chain(PDB_CHAIN)

for model in full_structure:
    for chain in model:
        if chain.id == PDB_CHAIN:
            for residue in chain:
                if residue.id[0] == ' ':
                    new_chain.add(residue.copy())
                elif residue.id[0] == f'H_{LIGAND_CODE}':
                    new_chain.add(residue.copy())

new_model.add(new_chain)
new_structure.add(new_model)

io = PDBIO()
io.set_structure(new_structure)
io.save(pdb_filename)

print(f"Downloaded and cleaned {pdb_filename}")

In [ ]:
# Separate protein and ligand
class NonHetSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.id[0] == ' ' else 0

class ResSelect(Select):
    def __init__(self, resname):
        self.resname = resname
    def accept_residue(self, residue):
        return 1 if residue.get_resname() == self.resname else 0

structure = parser.get_structure(TARGET_PDB_ID, pdb_filename)
io = PDBIO()
io.set_structure(structure)

protein_file = f'protein-{TARGET_PDB_ID}.pdb'
ligand_file = f'ligand-{LIGAND_CODE}.pdb'

io.save(protein_file, NonHetSelect())
io.save(ligand_file, ResSelect(LIGAND_CODE))

# Add hydrogens with OpenBabel
prepped_protein_file = f'{TARGET_PDB_ID}_prepped.pdb'
!obabel {protein_file} -O {prepped_protein_file} -h 2>/dev/null

print(f"Protein prepared: {prepped_protein_file}")
print(f"Reference ligand: {ligand_file}")

## Step 2: Define the molecules to dock

We provide molecules as **SMILES** strings (a text-based way to describe molecular structures) and convert them to 3D coordinates.

We'll dock the same molecules we predicted in Notebook 2:
- 4-hydroxytamoxifen (the reference from the crystal structure)
- Estradiol (natural hormone)
- Tamoxifen (pro-drug)
- Raloxifene (another ERα drug)

In [ ]:
test_smiles = [
    'OC1=CC=C(/C(=C(/CC)C2=CC=CC=C2)C3=CC=C(OCCN(C)C)C=C3)C=C1',  # 4-Hydroxytamoxifen
    'O[C@@H]1CC[C@@H]2[C@H]3CCC4=CC(=CC=C4[C@@H]3CC[C@]12C)O',    # Estradiol
    'CCC(/C1=CC=CC=C1)=C(\\C2=CC=C(OCCN(C)C)C=C2)C3=CC=CC=C3',      # Tamoxifen
    'OC1=CC=C(C(=O)C2=CC=C(OCCN3CCCCC3)C=C2)C=C1/C(=C/4\\SC5=CC=C(O)C=C5)C4=O',  # Raloxifene
]

molecule_names = ['4-Hydroxytamoxifen', 'Estradiol', 'Tamoxifen', 'Raloxifene']

In [ ]:
# Generate 3D structures from SMILES
IDs = [f'ligand_{i:02d}' for i in range(len(test_smiles))]
df = pd.DataFrame({'ID': IDs, 'Name': molecule_names, 'SMILES': test_smiles})

df['rdkit_mol'] = [Chem.MolFromSmiles(s) for s in df['SMILES']]

# Show 2D images of the molecules
img = Draw.MolsToGridImage(
    df['rdkit_mol'].tolist(),
    molsPerRow=4,
    subImgSize=(250, 200),
    legends=df['Name'].tolist()
)

# Save as MOL files with 3D coordinates
for i in range(len(df)):
    mol = df.iloc[i]['rdkit_mol']
    name = df.iloc[i]['ID'] + '.mol'
    Chem.MolToMolFile(mol, name)

img

## Step 3: Define the docking box

We need to tell Vina *where* to dock the molecule. We define a 3D box around the binding site using:
- The **center of geometry** of the reference ligand (where to look)
- The **radius of gyration** to determine the box size (how big the search area should be)

> **What's happening here?** Think of it as putting a box around the keyhole so the computer knows where to try fitting the key, instead of searching the entire lock.

In [ ]:
def calculate_rg(filename):
    """Calculate the Radius of Gyration of a structure from a PDB file."""
    coord = []
    mass = []
    with open(filename, 'r') as f:
        for line in f:
            try:
                parts = line.split()
                x, y, z = float(parts[6]), float(parts[7]), float(parts[8])
                coord.append([x, y, z])
                element = parts[-1]
                mass_map = {'C': 12.011, 'O': 15.999, 'N': 14.007, 'S': 32.065, 'H': 1.008}
                mass.append(mass_map.get(element, 12.011))
            except:
                pass
    xm = [(m*i, m*j, m*k) for (i, j, k), m in zip(coord, mass)]
    tmass = sum(mass)
    rr = sum(mi*i + mj*j + mk*k for (i, j, k), (mi, mj, mk) in zip(coord, xm))
    mm = sum((sum(i) / tmass)**2 for i in zip(*xm))
    return round(math.sqrt(rr / tmass - mm), 3)


def calculate_cog(pdbfile):
    """Calculate the Center of Geometry of a structure from a PDB file."""
    coordinates = []
    with open(pdbfile) as f:
        for line in f:
            if line.startswith(('ATOM', 'HETATM')):
                coordinates.append([
                    float(line[30:38]),
                    float(line[38:46]),
                    float(line[46:54])
                ])
    center = [sum(c[j] for c in coordinates) / len(coordinates) for j in range(3)]
    return [round(c, 3) for c in center]


# Calculate box parameters from the reference ligand
rg = calculate_rg(ligand_file)
cog = calculate_cog(ligand_file)
box_size = round(rg * 2.9, 3)

print(f"Radius of Gyration: {rg} \u00c5")
print(f"Center of Geometry: {cog}")
print(f"Docking box size:   {box_size} \u00c5 in each dimension")

## Step 4: Prepare protein and ligands for docking

AutoDock Vina uses a special file format called **PDBQT** that includes atomic charges and atom types. We need to convert our protein and ligand files.

> **What's happening here?** The PDBQT format adds extra information about each atom: its partial electric charge (Q) and its type (T). Vina needs this to calculate how well a molecule binds.

In [ ]:
# --- PDBQT conversion functions ---
# These are needed by AutoDock Vina to understand the molecules

def PDBQTAtomLines(mol, donors, acceptors):
    """Create PDBQT atom lines with proper atom types and charges."""
    atom_lines = [line.replace('HETATM', 'ATOM  ')
                  for line in Chem.MolToPDBBlock(mol).split('\n')
                  if line.startswith('HETATM') or line.startswith('ATOM')]

    pdbqt_lines = []
    for idx, atom in enumerate(mol.GetAtoms()):
        pdbqt_line = atom_lines[idx][:56]
        pdbqt_line += '0.00  0.00    '
        charge = 0.
        for f in ['_MMFF94Charge', '_GasteigerCharge', '_TriposPartialCharge']:
            if atom.HasProp(f):
                charge = atom.GetDoubleProp(f)
                break
        if isnan(charge) or isinf(charge):
            charge = 0.
        pdbqt_line += ('%.3f' % charge).rjust(6)
        pdbqt_line += ' '
        atomicnum = atom.GetAtomicNum()
        if atomicnum == 6 and atom.GetIsAromatic():
            pdbqt_line += 'A'
        elif atomicnum == 7 and idx in acceptors:
            pdbqt_line += 'NA'
        elif atomicnum == 8 and idx in acceptors:
            pdbqt_line += 'OA'
        elif atomicnum == 1 and atom.GetNeighbors()[0].GetIdx() in donors:
            pdbqt_line += 'HD'
        else:
            pdbqt_line += atom.GetSymbol()
        pdbqt_lines.append(pdbqt_line)
    return pdbqt_lines


def MolToPDBQTBlock(mol, flexible=True, addHs=False, computeCharges=False):
    """Convert an RDKit Molecule to a PDBQT block string."""
    mol = Chem.Mol(mol)
    if flexible and len(Chem.GetMolFrags(mol)) > 1:
        return ''.join(MolToPDBQTBlock(frag, flexible=flexible, addHs=addHs,
                                       computeCharges=computeCharges)
                       for frag in Chem.GetMolFrags(mol, asMols=True))

    # Identify donors and acceptors
    patt = Chem.MolFromSmarts('[$([O;H1;v2]),'
                              '$([O;H0;v2;!$(O=N-*),'
                              '$([O;-;!$(*-N=O)]),'
                              '$([o;+0])]),'
                              '$([n;+0;!X3;!$([n;H1](cc)cc),'
                              '$([$([N;H0]#[C&v4])]),'
                              '$([N&v3;H0;$(Nc)])]),'
                              '$([F;$(F-[#6]);!$(FC[F,Cl,Br,I])])]')
    acceptors = list(map(lambda x: x[0],
                         mol.GetSubstructMatches(patt, maxMatches=mol.GetNumAtoms())))
    patt = Chem.MolFromSmarts('[$([N&!H0&v3,N&!H0&+1&v4,n&H1&+0,$([$([Nv3](-C)(-C)-C)]),'
                              '$([$(n[n;H1]),'
                              '$(nc[n;H1])])]),'
                              '$([NX3,NX2]([!O,!S])!@C(!@[NX3,NX2]([!O,!S]))!@[NX3,NX2]([!O,!S])),'
                              '$([O,S;H1;+0])]')
    donors = list(map(lambda x: x[0],
                      mol.GetSubstructMatches(patt, maxMatches=mol.GetNumAtoms())))
    if addHs:
        mol = Chem.AddHs(mol, addCoords=True, onlyOnAtoms=donors)
    if addHs or computeCharges:
        AllChem.ComputeGasteigerCharges(mol)

    atom_lines = PDBQTAtomLines(mol, donors, acceptors)
    assert len(atom_lines) == mol.GetNumAtoms()

    pdbqt_lines = []
    if (mol.HasProp('vina_affinity') and mol.HasProp('vina_rmsd_lb') and
            mol.HasProp('vina_rmsd_lb')):
        pdbqt_lines.append('REMARK VINA RESULT:  ' +
                           ('%.1f' % float(mol.GetProp('vina_affinity'))).rjust(8) +
                           ('%.3f' % float(mol.GetProp('vina_rmsd_lb'))).rjust(11) +
                           ('%.3f' % float(mol.GetProp('vina_rmsd_ub'))).rjust(11))
    pdbqt_lines.append('REMARK  Name = ' +
                       (mol.GetProp('_Name') if mol.HasProp('_Name') else ''))
    if flexible:
        rot_bond = Chem.MolFromSmarts('[!$(*#*)&!D1&!$(C(F)(F)F)&'
                                      '!$(C(Cl)(Cl)Cl)&'
                                      '!$(C(Br)(Br)Br)&'
                                      '!$(C([CH3])([CH3])[CH3])&'
                                      '!$([CD3](=[N,O,S])-!@[#7,O,S!D1])&'
                                      '!$([#7,O,S!D1]-!@[CD3]=[N,O,S])&'
                                      '!$([CD3](=[N+])-!@[#7!D1])&'
                                      '!$([#7!D1]-!@[CD3]=[N+])]-!@[!$(*#*)&'
                                      '!D1&!$(C(F)(F)F)&'
                                      '!$(C(Cl)(Cl)Cl)&'
                                      '!$(C(Br)(Br)Br)&'
                                      '!$(C([CH3])([CH3])[CH3])]')
        bond_atoms = list(mol.GetSubstructMatches(rot_bond))
        num_torsions = len(bond_atoms)
        pdbqt_lines.append('REMARK  %i active torsions:' % num_torsions)
        pdbqt_lines.append('REMARK  status: (\'A\' for Active; \'I\' for Inactive)')
        for i, (a1, a2) in enumerate(bond_atoms):
            pdbqt_lines.append('REMARK%5.0i  A    between atoms: _%i  and  _%i'
                               % (i + 1, a1 + 1, a2 + 1))
        bond_ids = [mol.GetBondBetweenAtoms(a1, a2).GetIdx() for a1, a2 in bond_atoms]
        if bond_ids:
            mol_rigid_frags = Chem.FragmentOnBonds(mol, bond_ids, addDummies=False)
        else:
            mol_rigid_frags = mol
        frags = list(Chem.GetMolFrags(mol_rigid_frags))

        def weigh_frags(frag):
            num_bonds = sum(1 for a1, a2 in bond_atoms if a1 in frag or a2 in frag)
            return -len(frag), -num_bonds
        frags = sorted(frags, key=weigh_frags)

        pdbqt_lines.append('ROOT')
        frag = frags.pop(0)
        for idx in frag:
            pdbqt_lines.append(atom_lines[idx])
        pdbqt_lines.append('ENDROOT')

        branch_queue = []
        current_root = frag
        old_roots = [frag]
        visited_frags = []
        visited_bonds = []
        while len(frags) > len(visited_frags):
            end_branch = True
            for frag_num, frag in enumerate(frags):
                for bond_num, (a1, a2) in enumerate(bond_atoms):
                    if (frag_num not in visited_frags and
                        bond_num not in visited_bonds and
                        (a1 in current_root and a2 in frag or
                         a2 in current_root and a1 in frag)):
                        if a1 in current_root:
                            bond_dir = '%i %i' % (a1 + 1, a2 + 1)
                        else:
                            bond_dir = '%i %i' % (a2 + 1, a1 + 1)
                        pdbqt_lines.append('BRANCH %s' % bond_dir)
                        for idx in frag:
                            pdbqt_lines.append(atom_lines[idx])
                        branch_queue.append('ENDBRANCH %s' % bond_dir)
                        old_roots.append(current_root)
                        current_root = frag
                        visited_frags.append(frag_num)
                        visited_bonds.append(bond_num)
                        end_branch = False
                        break
                    else:
                        continue
                    break
            if end_branch:
                pdbqt_lines.append(branch_queue.pop())
                if old_roots:
                    current_root = old_roots.pop()
        while len(branch_queue):
            pdbqt_lines.append(branch_queue.pop())
        pdbqt_lines.append('TORSDOF %i' % num_torsions)
    else:
        pdbqt_lines.extend(atom_lines)

    return '\n'.join(pdbqt_lines)


def MolFromPDBQTBlock(filename, sanitize=True, removeHs=True):
    """Read a PDBQT file and return an RDKit Molecule."""
    pdb_lines = []
    name = ''
    data = {}
    with open(filename) as file:
        block = [line.rstrip() for line in file.readlines()]
    for line in block:
        if line[:12] == 'REMARK  Name':
            name = line[15:].strip()
        elif line[:18] == 'REMARK VINA RESULT':
            tmp = line[19:].split()
            data['vina_affinity'] = tmp[0]
            data['vina_rmsd_lb'] = tmp[1]
            data['vina_rmsd_ub'] = tmp[2]
        if line[:4] != 'ATOM':
            continue
        pdb_line = line[:56]
        pdb_line += '1.00  0.00           '
        atom_type = line[71:].split()[1]
        if atom_type == 'A':
            atom_type = 'C'
        elif atom_type[:1] == 'O':
            atom_type = 'O'
        elif atom_type[:1] == 'H':
            atom_type = 'H'
            if removeHs:
                continue
        elif atom_type == 'NA':
            atom_type = 'N'
        pdb_lines.append(pdb_line + atom_type)

    mol = Chem.MolFromPDBBlock('\n'.join(pdb_lines), sanitize=False)
    if sanitize:
        Chem.SanitizeMol(mol)
    else:
        Chem.GetSSSR(mol)
    new_order = sorted(range(mol.GetNumAtoms()),
                       key=lambda i: (mol.GetAtomWithIdx(i)
                                      .GetPDBResidueInfo()
                                      .GetSerialNumber()))
    mol = Chem.RenumberAtoms(mol, new_order)
    mol.SetProp('_Name', name)
    for k, v in data.items():
        mol.SetProp(str(k), str(v))
    return mol

print("PDBQT conversion functions loaded!")

In [ ]:
# Convert protein to PDBQT using OpenBabel
# (RDKit struggles with protein PDB files after hydrogen addition, so we use OpenBabel instead)
!obabel {prepped_protein_file} -O protein.pdbqt -xr 2>/dev/null

print("Protein converted to PDBQT format")

## Step 5: Dock the molecules!

Now we dock each molecule into the ERα binding site. This is the main event!

> **What's happening here?** For each molecule, Vina uses a search algorithm to explore different positions, orientations, and conformations (shapes) inside the docking box. It scores each pose based on physical interactions (hydrogen bonds, hydrophobic contacts, etc.) and returns the best one. The score (in kcal/mol) tells us the predicted binding strength — more negative = stronger binding.

In [ ]:
# Dock all molecules and collect results
docking_results = []

for i, row in df.iterrows():
    ligand_id = row['ID']
    name = row['Name']
    print(f"\nDocking {name} ({ligand_id})...")

    # Convert ligand to PDBQT
    m = Chem.MolFromMolFile(f'{ligand_id}.mol')
    ligand_pdbqt = MolToPDBQTBlock(m)
    with open(f'{ligand_id}.pdbqt', 'w') as f:
        f.write(ligand_pdbqt)

    # Run Vina docking
    v = Vina(sf_name='vina', seed=1234, verbosity=0)
    v.set_receptor('protein.pdbqt')
    v.set_ligand_from_file(f'{ligand_id}.pdbqt')
    v.compute_vina_maps(
        center=cog,
        box_size=[box_size, box_size, box_size]
    )
    v.dock(exhaustiveness=10, n_poses=10)
    v.write_poses(f'{ligand_id}_docked.pdbqt', n_poses=1, overwrite=True)

    # Extract results
    mol_result = MolFromPDBQTBlock(f'{ligand_id}_docked.pdbqt', sanitize=False)
    Chem.rdmolfiles.MolToPDBFile(mol_result, f'{ligand_id}_docked.pdb')

    affinity = float(mol_result.GetProp('vina_affinity'))
    Ki = math.exp(affinity / 0.592)
    pChEMBL = -math.log10(Ki)

    docking_results.append({
        'Name': name,
        'ID': ligand_id,
        'Vina_affinity_kcal': affinity,
        'pChEMBL_docking': round(pChEMBL, 2)
    })

    print(f"  Vina affinity: {affinity:.1f} kcal/mol")
    print(f"  Estimated pChEMBL: {pChEMBL:.2f}")

results_df = pd.DataFrame(docking_results)
print("\n\nDocking complete!")

## Step 6: Visualize the docking results

Let's see how the docked molecules look inside the binding site! The **reference ligand** (from the crystal structure) is shown in **magenta**, and the **docked pose** is shown in **cyan**.

In [ ]:
# Visualize the first docked molecule (4-hydroxytamoxifen) vs reference
LIGAND_TO_SHOW = 'ligand_00'  # Change this to view other molecules

view = py3Dmol.view(width=800, height=500)
view.removeAllModels()
view.setViewStyle({'style': 'outline', 'color': 'black', 'width': 0.1})

# Protein
view.addModel(open(prepped_protein_file, 'r').read(), format='pdb')
prot = view.getModel()
prot.setStyle({'cartoon': {'arrows': True, 'tubes': True, 'style': 'oval', 'color': 'white'}})
view.addSurface(py3Dmol.VDW, {'opacity': 0.6, 'color': 'white'})

# Reference ligand (magenta)
view.addModel(open(ligand_file, 'r').read(), format='pdb')
ref = view.getModel()
ref.setStyle({}, {'stick': {'colorscheme': 'magentaCarbon', 'radius': 0.2}})

# Docked ligand (cyan)
view.addModel(open(f'{LIGAND_TO_SHOW}_docked.pdb', 'r').read(), format='pdb')
docked = view.getModel()
docked.setStyle({}, {'stick': {'colorscheme': 'cyanCarbon', 'radius': 0.2}})

print('Reference: Magenta | Docked pose: Cyan')

view.zoomTo()
view.show()

## Step 7: Show binding site interactions

Let's look at which protein residues are close to the docked molecule.

In [ ]:
def get_binding_site_residues(pdb_file, ligand_code, radius=5.0):
    """Find protein residues within radius of a ligand using BioPython."""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('complex', pdb_file)
    model = structure[0]

    ligand_atoms = []
    protein_atoms = []
    for residue in model.get_residues():
        if residue.get_resname() == ligand_code or residue.id[0].startswith('H_'):
            ligand_atoms.extend(residue.get_atoms())
        elif is_aa(residue):
            protein_atoms.extend(residue.get_atoms())

    if not ligand_atoms or not protein_atoms:
        # For docked complexes, the ligand is UNL
        ligand_atoms = []
        protein_atoms = []
        for residue in model.get_residues():
            if residue.get_resname() == 'UNL' or residue.id[0].startswith('H_'):
                ligand_atoms.extend(residue.get_atoms())
            elif is_aa(residue):
                protein_atoms.extend(residue.get_atoms())

    if not ligand_atoms:
        print("Could not find ligand atoms!")
        return []

    ns = NeighborSearch(protein_atoms)
    nearby_atoms = set()
    for atom in ligand_atoms:
        neighbors = ns.search(atom.coord, radius, level='A')
        nearby_atoms.update(neighbors)

    nearby_residues = set(atom.get_parent() for atom in nearby_atoms)
    result = []
    for res in sorted(nearby_residues, key=lambda r: r.id[1]):
        result.append({'resid': res.id[1], 'resname': res.get_resname(), 'chain': res.get_parent().id})
    return result

In [ ]:
# Combine protein + docked ligand into a complex for analysis
complex_file = f'{TARGET_PDB_ID}-docked-complex.pdb'
with open(complex_file, 'w') as outfile:
    for fname in [prepped_protein_file, f'{LIGAND_TO_SHOW}_docked.pdb']:
        with open(fname) as infile:
            for line in infile:
                if 'END' not in line:
                    outfile.write(line)

# Find nearby residues
binding_residues = get_binding_site_residues(complex_file, 'UNL')
print(f"Residues within 5 \u00c5 of the docked ligand:")
for res in binding_residues:
    print(f"  {res['resname']} {res['resid']}")

In [ ]:
# Visualize the binding site around the docked molecule
with open(complex_file, 'r') as f:
    complex_data = f.read()

view = py3Dmol.view(width=800, height=500)
view.addModel(complex_data, 'pdb')

# Protein as transparent cartoon
view.setStyle({'hetflag': False},
              {'cartoon': {'color': 'lightblue', 'opacity': 0.4}})

# Docked ligand as sticks
view.setStyle({'resn': 'UNL'},
              {'stick': {'colorscheme': 'cyanCarbon', 'radius': 0.2}})

# Binding site residues
resi_list = [res['resid'] for res in binding_residues]
view.addStyle({'resi': resi_list, 'hetflag': False},
              {'stick': {'colorscheme': 'whiteCarbon', 'radius': 0.1}})

# Labels
for res in binding_residues:
    view.addResLabels({'resi': res['resid'], 'atom': 'CA'},
                      {'font': 'Arial', 'fontSize': 10,
                       'fontColor': 'black', 'backgroundColor': 'white',
                       'backgroundOpacity': 0.6})

view.zoomTo({'resn': 'UNL'})
view.show()

## Step 8: Compare docking results with ML predictions

Let's put the docking scores side by side with the ML predictions from Notebook 2.

> **What's happening here?** We used two completely different approaches to predict how well molecules bind: ML (pattern recognition from data) and docking (physics-based 3D fitting). If both methods agree, that gives us more confidence in the prediction!

In [ ]:
# ML predictions from Notebook 2 (you can update these with your actual values)
ml_predictions = {
    '4-Hydroxytamoxifen': 7.0,  # Update with your Notebook 2 results!
    'Estradiol': 7.5,
    'Tamoxifen': 6.5,
    'Raloxifene': 7.0,
}

print("Comparison: ML predictions vs Docking")
print("=" * 65)
print(f"{'Molecule':25s}  {'ML pChEMBL':>12s}  {'Dock pChEMBL':>13s}  {'Vina (kcal)':>11s}")
print("-" * 65)

for _, row in results_df.iterrows():
    ml_val = ml_predictions.get(row['Name'], '?')
    if isinstance(ml_val, (int, float)):
        ml_str = f"{ml_val:.2f}"
    else:
        ml_str = str(ml_val)
    print(f"{row['Name']:25s}  {ml_str:>12s}  {row['pChEMBL_docking']:>13.2f}  {row['Vina_affinity_kcal']:>11.1f}")

**Things to think about:**
- Do the ML and docking predictions agree?
- Which molecule is predicted to bind most strongly?
- Does that match what we know about these drugs in real life?

---

## Summary

In this notebook, you learned how to:

1. Prepare a protein for molecular docking
2. Generate 3D structures of drug molecules from SMILES
3. Define a docking box around the binding site
4. Run molecular docking with AutoDock Vina
5. Visualize docking results and binding site interactions
6. Compare docking results with machine learning predictions

---

## Try it yourself!

Want to dock additional molecules? Add their SMILES to the list below, run the cells, and see how they compare!

You can find SMILES for any molecule on [PubChem](https://pubchem.ncbi.nlm.nih.gov/) (search by name, then look for "Canonical SMILES") or draw your own with the [PubChem Sketcher](https://pubchem.ncbi.nlm.nih.gov/edit3/index.html).

In [ ]:
# --- TRY IT YOURSELF ---
# Add SMILES strings and names for molecules you'd like to dock

extra_smiles = [
    # ('Molecule Name', 'SMILES_STRING_HERE'),
]

for name, smi in extra_smiles:
    print(f"\nDocking {name}...")
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        print(f"  Could not parse SMILES: {smi}")
        continue

    Chem.MolToMolFile(mol, 'extra_ligand.mol')
    m = Chem.MolFromMolFile('extra_ligand.mol')
    pdbqt = MolToPDBQTBlock(m)
    with open('extra_ligand.pdbqt', 'w') as f:
        f.write(pdbqt)

    v = Vina(sf_name='vina', seed=1234, verbosity=0)
    v.set_receptor('protein.pdbqt')
    v.set_ligand_from_file('extra_ligand.pdbqt')
    v.compute_vina_maps(center=cog, box_size=[box_size, box_size, box_size])
    v.dock(exhaustiveness=10, n_poses=10)
    v.write_poses('extra_docked.pdbqt', n_poses=1, overwrite=True)

    result_mol = MolFromPDBQTBlock('extra_docked.pdbqt', sanitize=False)
    affinity = float(result_mol.GetProp('vina_affinity'))
    Ki = math.exp(affinity / 0.592)
    pChEMBL = -math.log10(Ki)
    print(f"  Vina affinity: {affinity:.1f} kcal/mol")
    print(f"  Estimated pChEMBL: {pChEMBL:.2f}")

if not extra_smiles:
    print("Add molecules to the extra_smiles list above and re-run!")